***Red Bus Automation Project***


scrape the data from RedBus Website & Push to Postgres SQL

In [ ]:
# Scrape the Redbus APSRTC GOVT BUS from website,  route : 'Hyderabad to Vijayawada.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Navigate to the RedBus website
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

try:
    # Hover over and click the APSRTC option
    hover_element = driver.find_element(By.CLASS_NAME, "rtcName")
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()
    time.sleep(3)

    apsrtc_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'rtcName') and text()='APSRTC']"))
    )
    apsrtc_option.click()
    time.sleep(5)

    # Select a route
    routes = driver.find_elements(By.XPATH, "//a[@class='route']")
    if not routes:
        print("No routes found.")
        driver.quit()
        exit()

    # Use the first route
    route_link = routes[0].get_attribute("href")
    route_name = routes[0].text
    driver.get(route_link)
    time.sleep(5)

    # Check for buses
    bus_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    if not bus_elements:
        print("No buses found. Attempting alternative method...")
        fallback_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="fixer"]/div/div/div[1]/span[3]/i'))
        )
        action.move_to_element(fallback_element).pause(1).click().perform()
        time.sleep(5)
        
    # Locate and hover over the 'rtcName' element
    hover_element = driver.find_element(By.CLASS_NAME, "button")
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()
    time.sleep(3)

    # Click on the 'APSRTC govt buses' option
    apsrtc_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'button') and text()='View Buses']"))
    )
    apsrtc_option.click()
    time.sleep(10)

    # Scrolling and extracting buses
    scroll_count = 0
    APSRTC_GOV_bus_data = []

    while scroll_count < 5:  # Scroll up to 5 times
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(7)
        scroll_count += 1

        # Fetch bus details
        #bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        #Use . Instead of text(): If the text content is nested or mixed, . can capture all text within the element:
        bus_names = driver.find_elements(By.XPATH, "//div[contains(@class, 'travels lh-24 f-bold d-color') and contains(., 'APSRTC')]")

        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        #star_ratings = driver.find_elements(By.XPATH, "//span[contains(text(), '.') and string-length(text()) > 2]")
        star_ratings = driver.find_elements(By.XPATH, "//span[normalize-space(.) != '' and contains(text(), '.') and number(text())]")#[:10]
        prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare')]/span[contains(@class, 'f-19')]")
        seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Find the minimum length to avoid mismatches
        min_length = min(len(bus_names), len(bus_types), len(departing_times), len(durations),
                         len(reaching_times), len(star_ratings), len(prices), len(seats_available))
        
                # Extract and format data
        for i in range(min_length):
            # Extract and process the bus_name
            raw_bus_name = bus_names[i].text
            bus_name_parts = raw_bus_name.split(' - ')  # Split 'APSRTC - 9363' into ['APSRTC', '9363']
            bus_company = bus_name_parts[0] if len(bus_name_parts) > 0 else None  # Extract 'APSRTC'
            bus_number = int(bus_name_parts[1]) if len(bus_name_parts) > 1 and bus_name_parts[1].isdigit() else None  # Extract and convert '9363' to number

            # Create the dictionary

            for i in range(min_length):
                bus_info = {
                    "route_name": route_name,
                    "route_link": route_link,
                    "bus_name": bus_names[i].text,
                    "bus_type": bus_types[i].text,
                    "departing_time": departing_times[i].text,
                    "duration": durations[i].text,
                    "reaching_time": reaching_times[i].text,
                    "star_rating": star_ratings[i].text,
                    "price": prices[i].text.replace('₹', '').strip(),
                    "seats_available": seats_available[i].text.replace('Seats available', '').strip(),
                }
                if bus_info not in APSRTC_GOV_bus_data:  # Avoid duplicates
                    APSRTC_GOV_bus_data.append(bus_info)

        print(f"Scroll {scroll_count}: Found {len(APSRTC_GOV_bus_data)} unique buses.")

        if len(APSRTC_GOV_bus_data) >= 10:  # Stop once 10 buses are collected
            break

    # Print the collected data
    print("\nCollected Bus Data:")
    for bus in APSRTC_GOV_bus_data[:10]:  # Limit output to 10 buses
        print(bus)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()
    


In [ ]:
# Govt bus APSRTC_GOV_bus_data Hydrebad Scrape the Redbus APSRTC GOVT BUS from website,  route : 'Hyderabad to Vijayawada.
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime

try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="red_bus",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Create the bus_routes table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_routes (
        id SERIAL PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        bus_name TEXT DEFAULT NULL,
        bus_type TEXT DEFAULT NULL,
        departing_time TIME DEFAULT NULL,
        duration TEXT DEFAULT NULL,
        reaching_time TIME DEFAULT NULL,
        star_rating FLOAT DEFAULT NULL,
        price DECIMAL DEFAULT NULL,
        seats_available INT DEFAULT NULL
    )
    """)

    # Insert data into the table
    for row in APSRTC_GOV_bus_data:
        cursor.execute("""
        INSERT INTO bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, 
        reaching_time, star_rating, price, seats_available) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['route_name'], row['route_link'], row['bus_name'], row['bus_type'],
            datetime.strptime(row['departing_time'], "%H:%M").time() if row['departing_time'] else None,
            row['duration'],
            datetime.strptime(row['reaching_time'], "%H:%M").time() if row['reaching_time'] else None,
            float(row['star_rating']) if row['star_rating'] else None,
            float(row['price']) if row['price'] else None,
            int(row['seats_available']) if row['seats_available'] else None
        ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
 # Scrape the Redbus APSRTC PRIVATE BUS from website,  route : 'Hyderabad to Vijayawada

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Navigate to the RedBus website
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

try:
    # Hover over and click the APSRTC option
    hover_element = driver.find_element(By.CLASS_NAME, "rtcName")
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()
    time.sleep(3)

    apsrtc_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'rtcName') and text()='APSRTC']"))
    )
    apsrtc_option.click()
    time.sleep(5)

    # Select a route
    routes = driver.find_elements(By.XPATH, "//a[@class='route']")
    if not routes:
        print("No routes found.")
        driver.quit()
        exit()

    # Use the first route
    route_link = routes[0].get_attribute("href")
    route_name = routes[0].text
    driver.get(route_link)
    time.sleep(5)

    # Check for buses
    bus_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    if not bus_elements:
        print("No buses found. Attempting alternative method...")
        fallback_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="fixer"]/div/div/div[1]/span[3]/i'))
        )
        action.move_to_element(fallback_element).pause(1).click().perform()
        time.sleep(5)

    # Scrolling and extracting buses
    scroll_count = 0
    Apsrtc_pvt_bus_data = []

    while scroll_count < 5:  # Scroll up to 5 times
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(7)
        scroll_count += 1

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_ratings = driver.find_elements(By.XPATH, "//span[contains(text(), '.') and string-length(text()) > 2]")
        prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare')]/span[contains(@class, 'f-19')]")
        seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Find the minimum length to avoid mismatches
        min_length = min(len(bus_names), len(bus_types), len(departing_times), len(durations),
                         len(reaching_times), len(star_ratings), len(prices), len(seats_available))

        for i in range(min_length):
            bus_info = {
                "route_name": route_name,
                "route_link": route_link,
                "bus_name": bus_names[i].text,
                "bus_type": bus_types[i].text,
                "departing_time": departing_times[i].text,
                "duration": durations[i].text,
                "reaching_time": reaching_times[i].text,
                "star_rating": star_ratings[i].text,
                "price": prices[i].text.replace('₹', '').strip(),
                "seats_available": seats_available[i].text.replace('Seats available', '').strip(),
            }
            if bus_info not in Apsrtc_pvt_bus_data :  # Avoid duplicates
                Apsrtc_pvt_bus_data .append(bus_info)

        print(f"Scroll {scroll_count}: Found {len(Apsrtc_pvt_bus_data )} unique buses.")

        if len(Apsrtc_pvt_bus_data ) >= 10:  # Stop once 10 buses are collected
            break

    # Print the collected data
    print("\nCollected Bus Data:")
    for bus in Apsrtc_pvt_bus_data[:10]:  # Limit output to 10 buses
        print(bus)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()


In [ ]:
#  insert the data Scrape the Redbus APSRTC PRIVATE BUS from website,  route : 'Hyderabad to Vijayawada
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime

try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="red_bus",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Create the bus_routes table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_routes (
        id SERIAL PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        bus_name TEXT DEFAULT NULL,
        bus_type TEXT DEFAULT NULL,
        departing_time TIME DEFAULT NULL,
        duration TEXT DEFAULT NULL,
        reaching_time TIME DEFAULT NULL,
        star_rating FLOAT DEFAULT NULL,
        price DECIMAL DEFAULT NULL,
        seats_available INT DEFAULT NULL
    )
    """)

    # Insert data into the table
    for row in Apsrtc_pvt_bus_data:
        cursor.execute("""
        INSERT INTO bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, 
        reaching_time, star_rating, price, seats_available) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['route_name'], row['route_link'], row['bus_name'], row['bus_type'],
            datetime.strptime(row['departing_time'], "%H:%M").time() if row['departing_time'] else None,
            row['duration'],
            datetime.strptime(row['reaching_time'], "%H:%M").time() if row['reaching_time'] else None,
            float(row['star_rating']) if row['star_rating'] else None,
            float(row['price']) if row['price'] else None,
            int(row['seats_available']) if row['seats_available'] else None
        ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
 # Scrape the Redbus APSRTC PRIVATE BUS from website,  route : Vijayawada to Hyderabad

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Navigate to the RedBus website
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

try:
    # Hover over and click the APSRTC option
    hover_element = driver.find_element(By.CLASS_NAME, "rtcName")
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()
    time.sleep(3)

    apsrtc_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'rtcName') and text()='APSRTC']"))
    )
    apsrtc_option.click()
    time.sleep(5)

    # Select a route
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
    
    routes = driver.find_elements(By.XPATH, "//a[contains(@class, 'route') and text()='Vijayawada to Hyderabad']")

    
    if not routes:
        print("No routes found.")
        driver.quit()
        exit()

    # Use the first route
    route_link = routes[0].get_attribute("href")
    route_name = routes[0].text
    driver.get(route_link)
    time.sleep(5)

    # Check for buses
    bus_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    if not bus_elements:
        print("No buses found. Attempting alternative method...")
        fallback_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="fixer"]/div/div/div[1]/span[3]/i'))
        )
        action.move_to_element(fallback_element).pause(1).click().perform()
        time.sleep(5)

    # Scrolling and extracting buses
    scroll_count = 0
    Apsrtc_pvt_bus_data = []

    while scroll_count < 5:  # Scroll up to 5 times
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(7)
        scroll_count += 1

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_ratings = driver.find_elements(By.XPATH, "//span[contains(text(), '.') and string-length(text()) > 2]")
        prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare')]/span[contains(@class, 'f-19')]")
        seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Find the minimum length to avoid mismatches
        min_length = min(len(bus_names), len(bus_types), len(departing_times), len(durations),
                         len(reaching_times), len(star_ratings), len(prices), len(seats_available))

        for i in range(min_length):
            bus_info = {
                "route_name": route_name,
                "route_link": route_link,
                "bus_name": bus_names[i].text,
                "bus_type": bus_types[i].text,
                "departing_time": departing_times[i].text,
                "duration": durations[i].text,
                "reaching_time": reaching_times[i].text,
                "star_rating": star_ratings[i].text,
                "price": prices[i].text.replace('₹', '').strip(),
                "seats_available": seats_available[i].text.replace('Seats available', '').strip(),
            }
            if bus_info not in Apsrtc_pvt_bus_data :  # Avoid duplicates
                Apsrtc_pvt_bus_data .append(bus_info)

        print(f"Scroll {scroll_count}: Found {len(Apsrtc_pvt_bus_data )} unique buses.")

        if len(Apsrtc_pvt_bus_data ) >= 10:  # Stop once 10 buses are collected
            break

    # Print the collected data
    print("\nCollected Bus Data:")
    for bus in Apsrtc_pvt_bus_data[:10]:  # Limit output to 10 buses
        print(bus)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()


In [ ]:
#  insert the data   Scrape the Redbus APSRTC PRIVATE BUS from website,  route : Vijayawada to Hyderabad
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime

try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="red_bus",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Create the bus_routes table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_routes (
        id SERIAL PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        bus_name TEXT DEFAULT NULL,
        bus_type TEXT DEFAULT NULL,
        departing_time TIME DEFAULT NULL,
        duration TEXT DEFAULT NULL,
        reaching_time TIME DEFAULT NULL,
        star_rating FLOAT DEFAULT NULL,
        price DECIMAL DEFAULT NULL,
        seats_available INT DEFAULT NULL
    )
    """)

    # Insert data into the table
    for row in Apsrtc_pvt_bus_data:
        cursor.execute("""
        INSERT INTO bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, 
        reaching_time, star_rating, price, seats_available) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['route_name'], row['route_link'], row['bus_name'], row['bus_type'],
            datetime.strptime(row['departing_time'], "%H:%M").time() if row['departing_time'] else None,
            row['duration'],
            datetime.strptime(row['reaching_time'], "%H:%M").time() if row['reaching_time'] else None,
            float(row['star_rating']) if row['star_rating'] else None,
            float(row['price']) if row['price'] else None,
            int(row['seats_available']) if row['seats_available'] else None
        ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
button_element = driver.find_element(By.CLASS_NAME, "button")
driver.execute_script("arguments[0].scrollIntoView();", button_element)
time.sleep(2)  # Allow time for scrolling
button_element.click()


In [ ]:
# Govt bus APSRTC_GOV_bus_data Hydrebad Scrape the Redbus APSRTC GOVT BUS from website,  route : Vijayawada to Hyderabad

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Navigate to the RedBus website
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

try:
    # Hover over and click the APSRTC option
    hover_element = driver.find_element(By.CLASS_NAME, "rtcName")
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()
    time.sleep(3)

    apsrtc_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'rtcName') and text()='APSRTC']"))
    )
    apsrtc_option.click()
    time.sleep(5)

    # Select a route
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
    
    routes = driver.find_elements(By.XPATH, "//a[contains(@class, 'route') and text()='Vijayawada to Hyderabad']")
    
    if not routes:
        print("No routes found.")
        driver.quit()
        exit()

    # Use the first route
    route_link = routes[0].get_attribute("href")
    route_name = routes[0].text
    driver.get(route_link)
    time.sleep(5)

    # Check for buses
    bus_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    if not bus_elements:
        print("No buses found. Attempting alternative method...")
        fallback_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="fixer"]/div/div/div[1]/span[3]/i'))
        )
        action.move_to_element(fallback_element).pause(1).click().perform()
        time.sleep(5)
    
    # Locate and hover over the 'rtcName' element
    hover_element = driver.find_element(By.XPATH, "//div[contains(@class, 'f-bold grop-name m-top-20') and text()='APSRTC Buses']")
    button_element = driver.find_element(By.CLASS_NAME, "button")

    # Scroll into view before clicking
    driver.execute_script("arguments[0].scrollIntoView();", button_element)
    time.sleep(2)
    button_element.click()
    
    time.sleep(10)

    # Scrolling and extracting buses
    scroll_count = 0
    APSRTC_GOV_bus_data = []

    while scroll_count < 5:  # Scroll up to 5 times
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(7)
        scroll_count += 1

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[contains(@class, 'travels lh-24 f-bold d-color') and contains(., 'APSRTC')]")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_ratings = driver.find_elements(By.XPATH, "//span[normalize-space(.) != '' and contains(text(), '.') and number(text())]")
        prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare')]/span[contains(@class, 'f-19')]")
        seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Find the minimum length to avoid mismatches
        min_length = min(len(bus_names), len(bus_types), len(departing_times), len(durations),
                         len(reaching_times), len(star_ratings), len(prices), len(seats_available))
        
        # Extract and format data
        for i in range(min_length):
            bus_info = {
                "route_name": route_name,
                "route_link": route_link,
                "bus_name": bus_names[i].text,
                "bus_type": bus_types[i].text,
                "departing_time": departing_times[i].text,
                "duration": durations[i].text,
                "reaching_time": reaching_times[i].text,
                "star_rating": star_ratings[i].text,
                "price": prices[i].text.replace('₹', '').strip(),
                "seats_available": seats_available[i].text.replace('Seats available', '').strip(),
            }
            if bus_info not in APSRTC_GOV_bus_data:  # Avoid duplicates
                APSRTC_GOV_bus_data.append(bus_info)

        print(f"Scroll {scroll_count}: Found {len(APSRTC_GOV_bus_data)} unique buses.")

        if len(APSRTC_GOV_bus_data) >= 10:  # Stop once 10 buses are collected
            break

    # Print the collected data
    print("\nCollected Bus Data:")
    for bus in APSRTC_GOV_bus_data[:10]:  # Limit output to 10 buses
        print(bus)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

In [ ]:
#push into pg sql Govt bus APSRTC_GOV_bus_data Hydrebad Scrape the Redbus APSRTC GOVT BUS from website,  route : Vijayawada to Hyderabad
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime

try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="red_bus",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Reset the sequence before inserting new rows
    cursor.execute("SELECT setval('bus_routes_id_seq', COALESCE((SELECT MAX(id) FROM bus_routes), 1), true);")


    # Insert data into the table
    for row in APSRTC_GOV_bus_data:
        cursor.execute("""
        INSERT INTO bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, 
        reaching_time, star_rating, price, seats_available) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['route_name'], row['route_link'], row['bus_name'], row['bus_type'],
            datetime.strptime(row['departing_time'], "%H:%M").time() if row['departing_time'] else None,
            row['duration'],
            datetime.strptime(row['reaching_time'], "%H:%M").time() if row['reaching_time'] else None,
            float(row['star_rating']) if row['star_rating'] else None,
            float(row['price']) if row['price'] else None,
            int(row['seats_available']) if row['seats_available'] else None
        ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
# Scrape  the data  TGSRTC route :TGSRTC Gvt BUS : Hyderabad to Vijayawada.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

try:
    hover_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'rtcName') and text()='TGSRTC']"))
    )
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()

    TGSRTC_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'rtcName') and text()='TGSRTC']"))
    )
    TGSRTC_option.click()
    time.sleep(5)

    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)

    routes = driver.find_elements(By.XPATH, "//a[contains(@class, 'route') and text()='Hyderabad to Vijayawada']")

    if not routes:
        print("No routes found.")
        driver.quit()
        exit()

    route_link = routes[0].get_attribute("href")
    route_name = routes[0].text
    driver.get(route_link)
    time.sleep(5)

    fallback_element = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="fixer"]/div/div/div[1]/span[3]/i'))
    )
    action.move_to_element(fallback_element).pause(1).click().perform()
    time.sleep(5)

    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);") #scrolls to the middle of the page.

    tgsrtc_view_buses_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'TGSRTC Buses')]/parent::div/following-sibling::div//div[contains(@class, 'button')]"))
    )
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tgsrtc_view_buses_button)
    time.sleep(2)
    tgsrtc_view_buses_button.click()


    #hover_element = WebDriverWait(driver, 20).until(
    #EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'f-bold grop-name m-top-20') and (text()='TGSRTC Buses' or text()='View Buses')]"))
    #)

    #hover_element = WebDriverWait(driver, 20).until(
      # EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'f-bold grop-name m-top-20') and text()='TGSRTC Buses' and text()='View Buses']"))
    #)
    #action = ActionChains(driver)
    


    #hover_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, 
    #"//div[contains(@class, 'f-bold grop-name m-top-20') and text()='View Buses']")))

    #action = ActionChains(driver)
    #action.move_to_element(hover_element).perform()

    #button_element = WebDriverWait(driver, 20).until(
    #    EC.element_to_be_clickable((By.CLASS_NAME, "button"))
    #)

    #driver.execute_script("arguments[0].scrollIntoView();", button_element)
    #time.sleep(2)
    #button_element.click()

    #time.sleep(10)

    scroll_count = 0
    TGSRTC_GOV_bus_data = [] 

    while scroll_count < 5:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3) #reduced time.

        # Fetch bus details with explicit wait.
        bus_names = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'travels lh-24 f-bold d-color') and contains(., 'TGSRTC')]")))

        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_ratings = driver.find_elements(By.XPATH, "//span[normalize-space(.) != '' and contains(text(), '.') and number(text())]")
        prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare')]/span[contains(@class, 'f-19')]")
        seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        min_length = min(len(bus_names), len(bus_types), len(departing_times), len(durations),
                         len(reaching_times), len(star_ratings), len(prices), len(seats_available))

        for i in range(min_length):
            bus_info = {
                "route_name": route_name,
                "route_link": route_link,
                "bus_name": bus_names[i].text,
                "bus_type": bus_types[i].text,
                "departing_time": departing_times[i].text,
                "duration": durations[i].text,
                "reaching_time": reaching_times[i].text,
                "star_rating": star_ratings[i].text,
                "price": prices[i].text.replace('₹', '').strip(),
                "seats_available": seats_available[i].text.replace('Seats available', '').strip(),
            }
            if bus_info not in TGSRTC_GOV_bus_data:
                TGSRTC_GOV_bus_data.append(bus_info)
                if len(TGSRTC_GOV_bus_data) >= 10: #check inside the for loop.
                    break #break inner loop.

        print(f"Scroll {scroll_count}: Found {len(TGSRTC_GOV_bus_data)} unique buses.")

        if len(TGSRTC_GOV_bus_data) >= 10:
            break

        scroll_count += 1

    print("\nCollected Bus Data:")
    for bus in TGSRTC_GOV_bus_data[:10]:
        print(bus)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

In [ ]:
# push  the data  TGSRTC route :TGSRTC GOVT BUS : Hyderabad to Vijayawada.
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime

try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="red_bus",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Create the bus_routes table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_routes (
        id SERIAL PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        bus_name TEXT DEFAULT NULL,
        bus_type TEXT DEFAULT NULL,
        departing_time TIME DEFAULT NULL,
        duration TEXT DEFAULT NULL,
        reaching_time TIME DEFAULT NULL,
        star_rating FLOAT DEFAULT NULL,
        price DECIMAL DEFAULT NULL,
        seats_available INT DEFAULT NULL
    )
    """)

    # Insert data into the table
    for row in TGSRTC_GOV_bus_data:
        cursor.execute("""
        INSERT INTO bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, 
        reaching_time, star_rating, price, seats_available) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['route_name'], row['route_link'], row['bus_name'], row['bus_type'],
            datetime.strptime(row['departing_time'], "%H:%M").time() if row['departing_time'] else None,
            row['duration'],
            datetime.strptime(row['reaching_time'], "%H:%M").time() if row['reaching_time'] else None,
            float(row['star_rating']) if row['star_rating'] else None,
            float(row['price']) if row['price'] else None,
            int(row['seats_available']) if row['seats_available'] else None
        ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:

# scarpe the TGSRTC route :TGSRTC pvt BUS : Hyderabad to Vijayawada

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Navigate to the RedBus website
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

try:
    # Hover over and click the TGSRTC option

    hover_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'rtcName') and text()='TGSRTC']"))
    )
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()

    TGSRTC_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'rtcName') and text()='TGSRTC']"))
    )
    TGSRTC_option.click()
    time.sleep(5)

    # Select a route
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
    
    routes = driver.find_elements(By.XPATH, "//a[contains(@class, 'route') and text()='Hyderabad to Vijayawada']")

    
    if not routes:
        print("No routes found.")
        driver.quit()
        exit()

    # Use the first route
    route_link = routes[0].get_attribute("href")
    route_name = routes[0].text
    driver.get(route_link)
    time.sleep(5)

    # Check for buses
    bus_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    if not bus_elements:
        print("No buses found. Attempting alternative method...")
        fallback_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="fixer"]/div/div/div[1]/span[3]/i'))
        )
        action.move_to_element(fallback_element).pause(1).click().perform()
        time.sleep(5)

    # Scrolling and extracting buses
    scroll_count = 0
    Tgsrtc_pvt_bus_data = []

    while scroll_count < 5:  # Scroll up to 5 times
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(7)
        scroll_count += 1

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_ratings = driver.find_elements(By.XPATH, "//span[contains(text(), '.') and string-length(text()) > 2]")
        prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare')]/span[contains(@class, 'f-19')]")
        seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Find the minimum length to avoid mismatches
        min_length = min(len(bus_names), len(bus_types), len(departing_times), len(durations),
                         len(reaching_times), len(star_ratings), len(prices), len(seats_available))

        for i in range(min_length):
            bus_info = {
                "route_name": route_name,
                "route_link": route_link,
                "bus_name": bus_names[i].text,
                "bus_type": bus_types[i].text,
                "departing_time": departing_times[i].text,
                "duration": durations[i].text,
                "reaching_time": reaching_times[i].text,
                "star_rating": star_ratings[i].text,
                "price": prices[i].text.replace('₹', '').strip(),
                "seats_available": seats_available[i].text.replace('Seats available', '').strip(),
            }
            if bus_info not in Tgsrtc_pvt_bus_data :  # Avoid duplicates
                Tgsrtc_pvt_bus_data .append(bus_info)

        print(f"Scroll {scroll_count}: Found {len(Tgsrtc_pvt_bus_data )} unique buses.")

        if len(Tgsrtc_pvt_bus_data ) >= 10:  # Stop once 10 buses are collected
            break

    # Print the collected data
    print("\nCollected Bus Data:")
    for bus in Tgsrtc_pvt_bus_data[:10]:  # Limit output to 10 buses
        print(bus)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()


In [ ]:
# push  the data  TGSRTC route :TGSRTC pvt BUS : Hyderabad to Vijayawada.
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime

try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="red_bus",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Create the bus_routes table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_routes (
        id SERIAL PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        bus_name TEXT DEFAULT NULL,
        bus_type TEXT DEFAULT NULL,
        departing_time TIME DEFAULT NULL,
        duration TEXT DEFAULT NULL,
        reaching_time TIME DEFAULT NULL,
        star_rating FLOAT DEFAULT NULL,
        price DECIMAL DEFAULT NULL,
        seats_available INT DEFAULT NULL
    )
    """)

    # Insert data into the table
    for row in Tgsrtc_pvt_bus_data:
        cursor.execute("""
        INSERT INTO bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, 
        reaching_time, star_rating, price, seats_available) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['route_name'], row['route_link'], row['bus_name'], row['bus_type'],
            datetime.strptime(row['departing_time'], "%H:%M").time() if row['departing_time'] else None,
            row['duration'],
            datetime.strptime(row['reaching_time'], "%H:%M").time() if row['reaching_time'] else None,
            float(row['star_rating']) if row['star_rating'] else None,
            float(row['price']) if row['price'] else None,
            int(row['seats_available']) if row['seats_available'] else None
        ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
# Scrape  the data  TGSRTC route :TGSRTC Gvt BUS : Hyderabad to Khammam.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

try:
    hover_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'rtcName') and text()='TGSRTC']"))
    )
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()

    TGSRTC_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'rtcName') and text()='TGSRTC']"))
    )
    TGSRTC_option.click()
    time.sleep(5)

    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)

    routes = driver.find_elements(By.XPATH, "//a[contains(@class, 'route') and text()='Hyderabad to Khammam']")

    if not routes:
        print("No routes found.")
        driver.quit()
        exit()

    route_link = routes[0].get_attribute("href")
    route_name = routes[0].text
    driver.get(route_link)
    time.sleep(5)

    fallback_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="fixer"]/div/div/div[1]/span[3]/i'))
    )
    action.move_to_element(fallback_element).pause(1).click().perform()
    time.sleep(5)

    hover_element = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'f-bold grop-name m-top-20') and text()='TGSRTC Buses']"))
    )

    button_element = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "button"))
    )

    driver.execute_script("arguments[0].scrollIntoView();", button_element)
    time.sleep(2)
    button_element.click()

    time.sleep(10)

    scroll_count = 0
    TGSRTC_GOV_bus_data = [] 

    while scroll_count < 5:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3) #reduced time.

        # Fetch bus details with explicit wait.
        bus_names = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'travels lh-24 f-bold d-color') and contains(., 'TGSRTC')]")))

        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_ratings = driver.find_elements(By.XPATH, "//span[normalize-space(.) != '' and contains(text(), '.') and number(text())]")
        prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare')]/span[contains(@class, 'f-19')]")
        seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        min_length = min(len(bus_names), len(bus_types), len(departing_times), len(durations),
                         len(reaching_times), len(star_ratings), len(prices), len(seats_available))

        for i in range(min_length):
            bus_info = {
                "route_name": route_name,
                "route_link": route_link,
                "bus_name": bus_names[i].text,
                "bus_type": bus_types[i].text,
                "departing_time": departing_times[i].text,
                "duration": durations[i].text,
                "reaching_time": reaching_times[i].text,
                "star_rating": star_ratings[i].text,
                "price": prices[i].text.replace('₹', '').strip(),
                "seats_available": seats_available[i].text.replace('Seats available', '').strip(),
            }
            if bus_info not in TGSRTC_GOV_bus_data:
                TGSRTC_GOV_bus_data.append(bus_info)
                if len(TGSRTC_GOV_bus_data) >= 10: #check inside the for loop.
                    break #break inner loop.

        print(f"Scroll {scroll_count}: Found {len(TGSRTC_GOV_bus_data)} unique buses.")

        if len(TGSRTC_GOV_bus_data) >= 10:
            break

        scroll_count += 1

    print("\nCollected Bus Data:")
    for bus in TGSRTC_GOV_bus_data[:10]:
        print(bus)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

In [ ]:
# push  the data  TGSRTC route :TGSRTC gvt BUS :Hyderabad to Khammam.
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime

try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="red_bus",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Create the bus_routes table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_routes (
        id SERIAL PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        bus_name TEXT DEFAULT NULL,
        bus_type TEXT DEFAULT NULL,
        departing_time TIME DEFAULT NULL,
        duration TEXT DEFAULT NULL,
        reaching_time TIME DEFAULT NULL,
        star_rating FLOAT DEFAULT NULL,
        price DECIMAL DEFAULT NULL,
        seats_available INT DEFAULT NULL
    )
    """)

    # Insert data into the table
    for row in TGSRTC_GOV_bus_data:
        cursor.execute("""
        INSERT INTO bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, 
        reaching_time, star_rating, price, seats_available) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['route_name'], row['route_link'], row['bus_name'], row['bus_type'],
            datetime.strptime(row['departing_time'], "%H:%M").time() if row['departing_time'] else None,
            row['duration'],
            datetime.strptime(row['reaching_time'], "%H:%M").time() if row['reaching_time'] else None,
            float(row['star_rating']) if row['star_rating'] else None,
            float(row['price']) if row['price'] else None,
            int(row['seats_available']) if row['seats_available'] else None
        ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:

# scarpe the TGSRTC route :TGSRTC pvt BUS : Hyderabad to Khammam

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Navigate to the RedBus website
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

try:
    # Hover over and click the TGSRTC option

    hover_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'rtcName') and text()='TGSRTC']"))
    )
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()

    TGSRTC_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'rtcName') and text()='TGSRTC']"))
    )
    TGSRTC_option.click()
    time.sleep(5)

    # Select a route
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
    
    routes = driver.find_elements(By.XPATH, "//a[contains(@class, 'route') and text()='Hyderabad to Khammam']")

    
    if not routes:
        print("No routes found.")
        driver.quit()
        exit()

    # Use the first route
    route_link = routes[0].get_attribute("href")
    route_name = routes[0].text
    driver.get(route_link)
    time.sleep(5)

    # Check for buses
    bus_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    if not bus_elements:
        print("No buses found. Attempting alternative method...")
        fallback_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="fixer"]/div/div/div[1]/span[3]/i'))
        )
        action.move_to_element(fallback_element).pause(1).click().perform()
        time.sleep(5)

    # Scrolling and extracting buses
    scroll_count = 0
    Tgsrtc_pvt_bus_data = []

    while scroll_count < 5:  # Scroll up to 5 times
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(7)
        scroll_count += 1

        # Fetch bus details
        bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_ratings = driver.find_elements(By.XPATH, "//span[contains(text(), '.') and string-length(text()) > 2]")
        prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare')]/span[contains(@class, 'f-19')]")
        seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Find the minimum length to avoid mismatches
        min_length = min(len(bus_names), len(bus_types), len(departing_times), len(durations),
                         len(reaching_times), len(star_ratings), len(prices), len(seats_available))

        for i in range(min_length):
            bus_info = {
                "route_name": route_name,
                "route_link": route_link,
                "bus_name": bus_names[i].text,
                "bus_type": bus_types[i].text,
                "departing_time": departing_times[i].text,
                "duration": durations[i].text,
                "reaching_time": reaching_times[i].text,
                "star_rating": star_ratings[i].text,
                "price": prices[i].text.replace('₹', '').strip(),
                "seats_available": seats_available[i].text.replace('Seats available', '').strip(),
            }
            if bus_info not in Tgsrtc_pvt_bus_data :  # Avoid duplicates
                Tgsrtc_pvt_bus_data .append(bus_info)

        print(f"Scroll {scroll_count}: Found {len(Tgsrtc_pvt_bus_data )} unique buses.")

        if len(Tgsrtc_pvt_bus_data ) >= 10:  # Stop once 10 buses are collected
            break

    # Print the collected data
    print("\nCollected Bus Data:")
    for bus in Tgsrtc_pvt_bus_data[:10]:  # Limit output to 10 buses
        print(bus)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()


In [ ]:
# push  the data  TGSRTC route :TGSRTC pvt BUS : Hyderabad to Khammam.
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime

try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="red_bus",
        user="postgres",
        password="sample12"
    )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = connection.cursor()

    # Create the bus_routes table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_routes (
        id SERIAL PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        bus_name TEXT DEFAULT NULL,
        bus_type TEXT DEFAULT NULL,
        departing_time TIME DEFAULT NULL,
        duration TEXT DEFAULT NULL,
        reaching_time TIME DEFAULT NULL,
        star_rating FLOAT DEFAULT NULL,
        price DECIMAL DEFAULT NULL,
        seats_available INT DEFAULT NULL
    )
    """)

    # Insert data into the table
    for row in Tgsrtc_pvt_bus_data:
        cursor.execute("""
        INSERT INTO bus_routes (route_name, route_link, bus_name, bus_type, departing_time, duration, 
        reaching_time, star_rating, price, seats_available) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['route_name'], row['route_link'], row['bus_name'], row['bus_type'],
            datetime.strptime(row['departing_time'], "%H:%M").time() if row['departing_time'] else None,
            row['duration'],
            datetime.strptime(row['reaching_time'], "%H:%M").time() if row['reaching_time'] else None,
            float(row['star_rating']) if row['star_rating'] else None,
            float(row['price']) if row['price'] else None,
            int(row['seats_available']) if row['seats_available'] else None
        ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred while interacting with the database: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
# Scrape  the data Kerala KSRTC route : Gvt BUS : Kozhikode to Ernakulam.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

try:
    # Hover over the KERALA RTC element

    hover_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'rtcName') and text()='KERALA RTC']"))
    )
    action = ActionChains(driver)
    action.move_to_element(hover_element).perform()

    # Click the KERALA RTC element using JavaScript
    KERALA_RTC_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'rtcName') and text()='KERALA RTC']"))
    )
    #KERALA_RTC_option.click()
    driver.execute_script("arguments[0].click();", KERALA_RTC_option) #use javascript click.

    time.sleep(5)

    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)

    routes = driver.find_elements(By.XPATH, "//a[contains(@class, 'route') and text()='Kozhikode to Ernakulam']")

    if not routes:
        print("No routes found.")
        driver.quit()
        exit()

    route_link = routes[0].get_attribute("href")
    route_name = routes[0].text
    driver.get(route_link)
    time.sleep(5)

    fallback_element = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="fixer"]/div/div/div[1]/span[3]/i'))
    )
    action.move_to_element(fallback_element).pause(1).click().perform()
    time.sleep(5)



    # Click on the 'KSRTC govt buses' option
    KSRTC_view_buses_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'button') and text()='View Buses']"))
    )
    KSRTC_view_buses_button.click()
    time.sleep(10)

    scroll_count = 0
    KSRTC_Kozhi_to_Erna__GOV_bus_data = [] 

    while scroll_count < 5:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3) #reduced time.

        # Fetch bus details with explicit wait.
        bus_names = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'travels lh-24 f-bold d-color') and contains(., 'KSRTC')]")))

        bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
        durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        reaching_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
        star_ratings = driver.find_elements(By.XPATH, "//span[normalize-space(.) != '' and contains(text(), '.') and number(text())]")
        prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare')]/span[contains(@class, 'f-19')]")
        seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        min_length = min(len(bus_names), len(bus_types), len(departing_times), len(durations),
                         len(reaching_times), len(star_ratings), len(prices), len(seats_available))

        for i in range(min_length):
            bus_info = {
                "route_name": route_name,
                "route_link": route_link,
                "bus_name": bus_names[i].text,
                "bus_type": bus_types[i].text,
                "departing_time": departing_times[i].text,
                "duration": durations[i].text,
                "reaching_time": reaching_times[i].text,
                "star_rating": star_ratings[i].text,
                "price": prices[i].text.replace('₹', '').strip(),
                "seats_available": seats_available[i].text.replace('Seats available', '').strip(),
            }
            if bus_info not in KSRTC_Kozhi_to_Erna__GOV_bus_data :
                KSRTC_Kozhi_to_Erna__GOV_bus_data .append(bus_info)
                if len(KSRTC_Kozhi_to_Erna__GOV_bus_data ) >= 10: #check inside the for loop.
                    break #break inner loop.

        print(f"Scroll {scroll_count}: Found {len(KSRTC_Kozhi_to_Erna__GOV_bus_data )} unique buses.")

        if len(KSRTC_Kozhi_to_Erna__GOV_bus_data ) >= 10:
            break

        scroll_count += 1

    print("\nCollected Bus Data:")
    for bus in KSRTC_Kozhi_to_Erna__GOV_bus_data[:10]:
        print(bus)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()